In [13]:
import requests
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd
from pprint import pprint
import time

In [3]:
with open('hot_json.json',newline='') as hot:
    data = json.load(hot)
    df = pd.json_normalize(data)

In [4]:
df['TSEQNO']

0       236581
1       236578
2       236571
3       236495
4       236493
         ...  
6265    187472
6266    180860
6267    175198
6268    173304
6269    156514
Name: TSEQNO, Length: 6270, dtype: int64

In [ ]:
url_test = 'https://www.hotcar.com.tw/app/main/CWA/CWA060.js'
headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
r=requests.get(url_test,headers=headers)
r.text

In [12]:
# for d in df['TSEQNO'][:1]:
#     url = f'https://www.hotcar.com.tw/CWA/CWA060.html?TSEQNO={d}'
# headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
#     dc = DesiredCapabilities.CHROME
#     dc[''] = { 'Table1':'Array' }
url = 'https://www.hotcar.com.tw/CWA/CWA060.html?TSEQNO=236581'
driver = webdriver.Chrome('../chromedriver')
driver.get(url)
time.sleep(3)
soup=BeautifulSoup(driver.page_source,'lxml')
soup.find()
    
driver.close()

<html lang="zh-tw"><head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=0" name="viewport"/>
<title>HOT大聯盟：TOYOTA ALTIS優質二手車</title>
<meta content="!" name="fragment"/>
<meta content="HOT大聯盟：TOYOTA ALTIS優質二手車" property="og:site_name"/>
<meta content="website" property="og:type"/>
<meta content="https://www.hotcar.com.tw/CWA/CWA060.html?TSEQNO=236581" property="og:url"/>
<meta content="https://www.hotcar.com.tw/UPLOAD/CW/HF1002/2601507_L.jpg" property="og:image"/>
<meta content="我在HOT大聯盟找到這台ALTIS ，2009年，價格只要19.6萬，物美價廉太生火!!" property="og:title"/>
<meta content="HOT,HOT大聯盟,HOT中古車,HOT認證中古車,好車大聯盟,HOT好車大聯盟" property="og:keywords"/>
<meta class="intro" content="&lt;p&gt;上暘汽車本著誠實.優質.信賴顧客為先.專業為本的服務精神為理念.上暘汽車給每個消費者在買賣中古車或維修保養時，一個值得信賴的好朋友，我們的特色包括:◎專業認證：HOT專業技術師人員查驗核發[認證書]◎三大保證：無泡水車、無事故車、無營業用車◎五大保固：引擎本體、變速箱本體、方向機本體、啟動馬達、發電機&lt;/p&gt;" property="og:description"/>
<meta content="HOT好

In [15]:
import urllib.request as req

In [40]:
url_try='https://www.hotcar.com.tw/SSAPI45/API/SPRetB?Token=VfaU+LJXyYZp7Nr3mFhCQtBfZ/rL2AQmOjkOW4W1uZVumEKn0wIHcD/RsdkmgB8di2Y9HFgUS/7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6'
url = 'https://www.hotcar.com.tw/CWA/CWA060.html?TSEQNO=236581'
requestdata={"SPNM":"CWA060Q_2018","SVRNM":["HOTCARAPP"],"PARMS":["236581","https://www.hotcar.com.tw","https://www.hotcar.com.tw/image/nophoto.png","",""]}
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
          'Content-Type':'application/json;charset=UTF-8',
          'Cookie':'_gcl_au=1.1.939139616.1624256679; _fbp=fb.2.1624256679494.1805338918; _ga=GA1.3.915183504.1624256680; __BWfp=c1624256679548xd6b06476d; clientID=915183504.1624256680; ASP.NET_SessionId=nrxwl2lrgxcsoaq4zw4jlvlk; Token=VfaU+LJXyYZp7Nr3mFhCQtBfZ/rL2AQmOjkOW4W1uZVumEKn0wIHcD/RsdkmgB8di2Y9HFgUS/7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6; _gid=GA1.3.1032427582.1624859448; Tagtoo_pta=pta_03+_&lta+_; _TUCI_T=sessionNumber+18806&pageView+18806&Search+18806; _TUCS=1; _TUCI=sessionNumber+5197&ECId+387&hostname+www.hotcar.com.tw&pageView+21786&Search+7000',
          'connection':'keep-alive',
          'Referer':'https://www.hotcar.com.tw/CWA/CWA060.html?TSEQNO=236581'}

# request = req.requests(url_try,headers=headers,data=json.dumps(requestdata).encode('utf-8'))

r=requests.post(url_try,headers=headers,files=requestdata)
r.encoding='utf-8'
r.text

ValueError: not enough values to unpack (expected 4, got 1)

In [ ]:
VfaU+LJXyYZp7Nr3mFhCQtBfZ/rL2AQmOjkOW4W1uZVumEKn0wIHcD/RsdkmgB8di2Y9HFgUS/7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6

In [ ]:
VfaU+LJXyYZp7Nr3mFhCQtBfZ/rL2AQmOjkOW4W1uZVumEKn0wIHcD/RsdkmgB8di2Y9HFgUS/7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6

In [ ]:
VfaU%2BLJXyYZp7Nr3mFhCQtBfZ%2FrL2AQmOjkOW4W1uZVumEKn0wIHcD%2FRsdkmgB8di2Y9HFgUS%2F7HFxHm4m9eACLvfBCTdBEGoGqcd6RDUeZNSwlOrVeFarS9bEalGyz6